# YugabyteDB: RF3 in Secure Mode

__NOTE: This notebook requires the Bash Kernel__

## Introduction

There are multiple options available to run YugabyteDB locally.  For simple, single node use cases, Docker is an exceptionally good tool.  However, for multi-node enviornments, Docker can be a bit trickier to configure correctly due to network setup.

The next option is to use the `yugabyted` command line tool outside Docker.  This tool is bundled with the YugabyteDB distribution and can be easily downloaded and unpacked to a local machine.  Typically, I recommend downloading the latest STS from the Yugabyte [downloads](https://download.yugabyte.com/#/) page.  Pick your preferred distribution (Mac or Linux) and download it with `curl`/`wget` as recommended and untar to a desired folder.

In [ ]:
# Initialize secrets from the local .env file here
source .env
echo $MY_SECRET

In [ ]:
mkdir ~/yugabyte; cd $_
#wget https://downloads.yugabyte.com/releases/2024.1.3.0/yugabyte-2024.1.3.0-b105-linux-x86_64.tar.gz
#tar xfz yugabyte-2024.1.3.0-b105-linux-x86_64.tar.gz
cd yugabyte-2024.1.3.0/
./bin/post_install.sh

## Validate the Installation

Assuming the previous commands ran successfully, you should have downloaded and unpacked the YugabyteDB distribution.  The final `post_install.sh` script is used to validate the deployent. The last output line should read, `INSTALL PASSED`.

Confirm the version running the `yugabyted` command with the `version` option:

In [ ]:
./bin/yugabyted version

## Generate Server Certificates

To run YugabyteDB locally with TLS (SSL) enabled, first create the certs for a 3 node REF cluster using [generate_server_certs](). This command will generate certs for each of the nodes using the IP address they will be running on.

On Linux we will use multiple local 127.0.0.x hosts:

In [ ]:
./bin/yugabyted cert generate_server_certs --hostnames=127.0.0.1,127.0.0.2,127.0.0.3

## Launch the first node

Before we start the first node of the cluster, lets set up a few enviornment variables that will help with configuring the YugabyteDB processes.
Review confiugration options here:

* [YB Master config](https://docs.yugabyte.com/preview/reference/configuration/yb-master/)
* [TServer config](https://docs.yugabyte.com/preview/reference/configuration/yb-tserver/)

In [ ]:
export MASTER_FLAGS=""
export TSERVER_FLAGS="yb_enable_read_committed_isolation=true"

For each of the three nodes we want to start, we use the `yugabyted` command with specific options to idenify each node (including the certs we generated earlier):

In [ ]:
export CERTS_DIR="~/var/generated_certs/127.0.0.1/"
yugabyted start --advertise_address=127.0.0.1 --base_dir=~/var/ybd-gcp1 --cloud_location=gcp.us-east1.us-east1-c --secure --certs_dir="${CERTS_DIR}" --tserver_flags "${TSERVER_FLAGS}"

## Retrieve the generated password

Take special note of the last line of the `yugabyted` output as it indicates where to find the generated password. Echo it out to the console:

In [ ]:
cat /home/lurtz/var/ybd-gcp1/data/yugabyted_credentials.txt

In [ ]:
export CERTS_DIR="~/var/generated_certs/127.0.0.2/"
yugabyted start --advertise_address=127.0.0.2 --base_dir=~/var/ybd-gcp2 --cloud_location=gcp.us-east1.us-east1-d --secure --certs_dir="${CERTS_DIR}" --tserver_flags "${TSERVER_FLAGS}" --join=127.0.0.1

In [ ]:
export CERTS_DIR="~/var/generated_certs/127.0.0.3/"
yugabyted start --advertise_address=127.0.0.3 --base_dir=~/var/ybd-gcp3 --cloud_location=gcp.us-east1.us-east1-f --secure --certs_dir="${CERTS_DIR}" --tserver_flags "${TSERVER_FLAGS}" --join=127.0.0.1

In [ ]:
yugabyted status --base_dir=~/var/ybd-gcp1

## Generate a new Java Trust Store (JKS)

For Java, use the `keytool` command (it should be bundled with the Java installation). The following command will create a new trust store (a special file) and import the root certificate authority (CA) from the orignal generated certs.

In [ ]:
keytool -importcert -trustcacerts -file ~/var/generated_certs/root-ca/ca.crt -alias yuabyteCA -keystore yugabyte.jks -storepass change1t -noprompt

Tecnically, you have a common truststore that is generically used by Java that contains standard CAs but since the CA we generated earlier is local and will probably be thrown away at some point, it is easier (and cleaner) to just keep a separate truststore for this specific use case.

In [ ]:
keytool -list -keystore yugabyte.jks -storepass change1t